# Disambiguation Pipeline: People

In [31]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")
import os

from heritageconnector.config import config

from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. import data from record matching step
At this point we have: 
- converted any URLs in our records to Wikidata references: `entity_matching.lookup`
- filtered Wikidata references for each record down to the exact person using their name, date of birth and date of death: `entity_matching.filtering`

In [7]:
df = pd.read_pickle("../../GITIGNORE_DATA/results/filtering_people_orgs_result.pkl")
df_people = df[df['GENDER'].isin(["M", "F"])]
df_people.loc[:, 'JOINED_NAME'] = df_people['FIRSTMID_NAME'] + " " + df_people['LASTSUFF_NAME']

df_people.head(2)

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered,JOINED_NAME
1,10245,"Zenthon, Edward Rupert",NaN,Edward Rupert,Zenthon,NaN,NaN,M,Y,REF: http://www.iwm.org.uk/collections/item/object/1030031461,nan,1920-07,"London, Greater London, England, United Kingdom",c. 2002,NaN,NaN,British,engineer,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,05-AUG-15,REF: http://www.iwm.org.uk/collections/item/object/1030031461 --- nan,[],[http://www.iwm.org.uk/collections/item/object/1030031461],[],Edward Rupert Zenthon
2,10269,"Troughton, John",NaN,John,Troughton,NaN,NaN,M,Y,"1739 - Born in Corney, Cumbria, England; Apprenticed to his Uncle John Troughton \n1764 - traded at Surrey St., Strand, London \n1768-71 - traded at Crown Court, Fleet St., London\n1771-78 - traded at 17 Dean St., Fetter Lane, London \n1778-82 - traded at 1 Queen's Sq., Bartholomew Close, London \n1782 - purchased the business of Benjamin Cole \n1782-1788 - traded at the sign of the Orrery, 136 Fleet St, London, England. \n1788-1804 - in partnership as J & E Troughton, with brother Edward Troughton (1756-1835)","ODNB: Anita McConnell, ‘Troughton, Edward (1753–1835)’, Oxford Dictionary of National Biography, Oxford University Press, 2004; online edn, May 2005 [http://www.oxforddnb.com/view/article/27767]\nREF: A. McConnell, Instrument makers to the world: a history of Cooke, Troughton & Simms (1992) · A. W. Skempton and J. Brown, ‘John and Edward Troughton’, Notes and Records of the Royal Society, 27 (1972–3), 233–62",1739,"Broughton in Furness, Cumbria, England, United Kingdom",1807,"London, Greater London, England, United Kingdom",NaN,English; British,mathematical instrument maker,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,06-NOV-18,"1739 - Born in Corney, Cumbria, England; Apprenticed to his Uncle John Troughton \n1764 - traded at Surrey St., Strand, London \n1768-71 - traded at Crown Court, Fleet St., London\n1771-78 - traded at 17 Dean St., Fetter Lane, London \n1778-82 - traded at 1 Queen's Sq., Bartholomew Close, London \n1782 - purchased the business of Benjamin Cole \n1782-1788 - traded at the sign of the Orrery, 136 Fleet St, London, England. \n1788-1804 - in partnership as J & E Troughton, with brother Edward Troughton (1756-1835) --- ODNB: Anita McConnell, ‘Troughton, Edward (1753–1835)’, Oxford Dictionary of National Biography, Oxford University Press, 2004; online edn, May 2005 [http://www.oxforddnb.com/view/article/27767]\nREF: A. McConnell, Instrument makers to the world: a history of Cooke, Troughton & Simms (1992) · A. W. Skempton and J. Brown, ‘John and Edward Troughton’, Notes and Records of the Royal Society, 27 (1972–3), 233–62",[Q1293897],[http://www.oxforddnb.com/view/article/27767],[],John Troughton


In [8]:
perc_matched_from_lookup = df_people['qcodes_filtered'].apply(len).value_counts()[1] / len(df_people)

print(f"{round(100*perc_matched_from_lookup, 1)}% of {len(df_people)} people have been matched unambiguously from the URL lookup step")

28.2% of 10352 people have been matched unambiguously from the URL lookup step


## 2. disambiguation

### 2.1 resolve categorical fields to Wikidata entities
This will be used to narrow down our searches which provide candidates for the disambiguator. For people, we will just look up **OCCUPATION**. Other potential candidates are GENDER, PLACE_OF_BIRTH and PLACE_OF_DEATH.

In [9]:
from heritageconnector.utils.data_transformation import transform_series_str_to_list
from heritageconnector.entity_matching.reconciler import reconciler

In [5]:
# create list column from OCCUPATION
df_people['OCCUPATION'] = transform_series_str_to_list(df_people['OCCUPATION'], separator=";")

# reconcile list column into list of qcodes
rec = reconciler(df_people, table="PEOPLE")
df_people["OCCUPATION_qcodes"] = rec.process_column("OCCUPATION", multiple_vals=True)

df_people.to_pickle("../../GITIGNORE_DATA/results/people_occupations_reconciled.pkl")

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
  0%|          | 0/2216 [00:00<?, ?it/s]

Looking up Wikidata qcodes for unique items..


  6%|▌         | 137/2216 [01:24<21:27,  1.61it/s]


KeyboardInterrupt: 

In [11]:
# else export from cached
# df_people = pd.read_pickle("../../GITIGNORE_DATA/results/people_occupations_reconciled.pkl")
df_people.loc[:, 'JOINED_NAME'] = df_people['FIRSTMID_NAME'] + " " + df_people['LASTSUFF_NAME'] # to remove once 2.1 is rerun

### 2.2 search - records for which we have at least one Wikidata ID for Occupations

In [13]:
from heritageconnector.disambiguation import search, retrieve

In [14]:
df_people_resolved = df_people[df_people["OCCUPATION_qcodes"].apply(len) > 0]

perc_resolved = len(df_people_resolved) / len(df_people)
num_without_match = len(set(df_people_resolved.index) - set(df_people[df_people['qcodes_filtered'].apply(len) > 0].index))

print(f"{round(perc_resolved*100, 2)}% of people have resolved occupations.")
print(f"This is {round(num_without_match/len(df_people)*100, 2)}%, excluding those who already have a match")

80.9% of people have resolved occupations.
This is 54.8%, excluding those who already have a match


#### Run search

In [35]:
len(df_people_resolved[df_people_resolved['qcodes_filtered'].apply(len) == 0])

5673

In [42]:
df_people_resolved.loc[:, "search_results"] = ""
df_search = df_people_resolved[df_people_resolved['qcodes_filtered'].apply(len) == 0].head(100)

for idx, row in tqdm(df_search.iterrows(), total=df_search.shape[0]):
    res = search.run(text=row["JOINED_NAME"], topn=3, limit=10, instanceof_filter="Q5", property_filters={"P106": row['OCCUPATION_qcodes']}).index.values.tolist()
    df_search.at[idx, "search_results"] = res

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
100%|██████████| 100/100 [17:05<00:00, 10.26s/it]


In [43]:
df_search[df_search['search_results'].apply(len) > 0]

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered,OCCUPATION_list,OCCUPATION_qcodes,JOINED_NAME,search_results
31,103617,"Job, Charles",NaN,Charles,Job,NaN,NaN,M,Y,RPS obituary.\nhttp://www.luminous-lint.com/app/photographer/Charles__Job/A/\n2010\nhttp://www.old-church-galleries.com/s_1600.asp,"Charles Job, a stockbroker by profession, was a keen amateur photographer, elected member of the Royal Photographic Society in 1893 and member of the Linked Ring Brotherhood. Worked at the Censor's Office in Liverpool during the first world war before returning to his native Sussex. Awarded Hon. FRPS in 1928. Noted for his landscape photographs.",1853,"Sussex, England, United Kingdom",1930,NaN,NaN,British,"[amateur photographer, stockbroker]",NaN,NaN,NaN,NaN,NaN,N,06-JAN-05,30-OCT-12,"RPS obituary.\nhttp://www.luminous-lint.com/app/photographer/Charles__Job/A/\n2010\nhttp://www.old-church-galleries.com/s_1600.asp --- Charles Job, a stockbroker by profession, was a keen amateur photographer, elected member of the Royal Photographic Society in 1893 and member of the Linked Ring Brotherhood. Worked at the Censor's Office in Liverpool during the first world war before returning to his native Sussex. Awarded Hon. FRPS in 1928. Noted for his landscape photographs.",[],"[http://www.luminous-lint.com/app/photographer/Charles__Job/A/, http://www.old-church-galleries.com/s_1600.asp]",[],"[amateur photographer, stockbroker]","[Q21694268, Q4182927]",Charles Job,[http://www.wikidata.org/entity/Q5075774]
33,10165,"Melville, Alexander",NaN,Alexander,Melville,NaN,NaN,M,Y,1878 - The London Gazette records Alexander Melville: 'improvements in machinery for producing ornamental designs on woollen fabrics'.,THE LONDON GAZETTE: https://www.thegazette.co.uk/London/issue/24540/page/135/data.pdf,NaN,NaN,1971,NaN,NaN,British,[inventor],NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,06-JUN-18,1878 - The London Gazette records Alexander Melville: 'improvements in machinery for producing ornamental designs on woollen fabrics'. --- THE LONDON GAZETTE: https://www.thegazette.co.uk/London/issue/24540/page/135/data.pdf,[],[https://www.thegazette.co.uk/London/issue/24540/page/135/data.pdf],[],[inventor],[Q205375],Alexander Melville,[http://www.wikidata.org/entity/Q34286]
40,11209,"Garratt, Colin",NaN,Colin,Garratt,NaN,NaN,M,N,http://www.milepost92-half.co.uk/ColinGarratt.htm,Colin Garratt is a photographer and author on steam railways.,NaN,NaN,NaN,NaN,NaN,British,"[railway photographer, author]",NaN,NaN,NaN,NaN,NaN,Y,11-FEB-98,31-AUG-17,http://www.milepost92-half.co.uk/ColinGarratt.htm --- Colin Garratt is a photographer and author on steam railways.,[],[http://www.milepost92-half.co.uk/ColinGarratt.htm],[],"[railway photographer, author]","[Q482980, Q15296811, Q36180]",Colin Garratt,[http://www.wikidata.org/entity/Q75727490]
56,11808,"Jaeger, Eduard",NaN,Eduard,Jaeger,NaN,NaN,M,Y,REF: http://fm.iowa.uiowa.edu/fmi/xsl/hardin/heirs/record_detail.xsl?-db=heirs&-lay=weblayout&HeirsNo=1977&-find\nWIKI: http://en.wikipedia.org/wiki/Eduard_J%C3%A4ger_von_Jaxtthal\nREF: http://www.aeiou.at/aeiou.encyclop.j/j072115.htm;internal&action=_setlanguage.action?LANGUAGE=en\nREF: http://beckerexhibits.wustl.edu/becker/records250.htm,"professor at the university of Vienna; used the ophthalmoscope for the determination of refractivity; made improvements to eye chart test types that were developed by Heinrich Kuechler; Ritter von Jaxtthal; published from 1844,",1818-06-25,"Vienna, Wien, Austria",1884-07-05,"Vienna, Wien, Austria",NaN,Austrian,[ophthalmologist],NaN,NaN,NaN,NaN,NaN,N,18-FEB-98,01-MAR-16,"REF: http://fm.iowa.uiowa.edu/fmi/xsl/hardin/heirs/record_detail.xsl?-db=heirs&-lay=weblayout&HeirsNo=